In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

%matplotlib inline
tf.config.set_visible_devices([], 'GPU')

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (150, 150))
            img_data = np.asarray(img)
            images.append(img_data)
    return images

In [3]:
def load_images_from_folder_2(folder):
    images = []
    for filename in os.listdir(folder):
        loaded_img = load_img(os.path.join(folder,filename), target_size=(150, 150))
        numpy_img = img_to_array(loaded_img)
        images.append(numpy_img)
    return images

In [4]:
control_images = load_images_from_folder_2('CT_NonCOVID')
covid_images = load_images_from_folder_2('CT_COVID')
all_images = control_images + covid_images
all_images = np.array(all_images)
# all_images = all_images.reshape((-1, 150, 150, 3))

In [5]:

all_images.shape

(746, 150, 150, 3)

In [6]:
# for i in range(4):
#     plt.subplot(2, 2, i + 1)
#     plt.imshow(np.uint8(all_images[i]))
# plt.show

In [7]:
control_labels = [0] * len(control_images)
covid_labels = [1] * len(covid_images)
label_series = pd.Series(control_labels + covid_labels)
print(len(all_images))
print(len(label_series))

746
746


In [8]:
X_train, X_test, y_train, y_test = train_test_split(all_images, label_series, test_size=0.25, random_state=42)

In [9]:
print(len(X_train))
print(len(y_train))

559
559


In [10]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

inp = tf.keras.layers.Input(shape=(150, 150, 3))
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inp)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
y = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inp, y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])


In [11]:
model.fit(X_train, y_train, batch_size=64, epochs=8, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()
print('COVID image detection accuracy:', ba.result().numpy())
print('COVID image detection recall:', recall_result)
print('COVID image detection precision:', precision_result)
print('COVID image detection F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 559 samples
Epoch 1/8
559/559 [==============================] - 18s 32ms/sample - loss: 10.2396 - accuracy: 0.4758
Epoch 2/8
559/559 [==============================] - 17s 31ms/sample - loss: 1.8671 - accuracy: 0.5116
Epoch 3/8
559/559 [==============================] - 18s 32ms/sample - loss: 1.0043 - accuracy: 0.5295
Epoch 4/8
559/559 [==============================] - 18s 32ms/sample - loss: 0.7355 - accuracy: 0.5635
Epoch 5/8
559/559 [==============================] - 18s 33ms/sample - loss: 0.6986 - accuracy: 0.6118
Epoch 6/8
559/559 [==============================] - 19s 33ms/sample - loss: 0.6041 - accuracy: 0.6601
Epoch 7/8
559/559 [==============================] - 18s 33ms/sample - loss: 0.5558 - accuracy: 0.7281
Epoch 8/8
559/559 [==============================] - 18s 33ms/sample - loss: 0.5491 - accuracy: 0.7156
COVID image detection accuracy: 0.6524064
COVID image detection recall: 0.9350649
COVID image detection precision: 0.54545456
COVID image detection F1 sco

In [12]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

inp = tf.keras.layers.Input(shape=(150, 150, 3))
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inp)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
y = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inp, y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [13]:
model.fit(X_train, y_train, batch_size=64, epochs=8, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()
print('COVID image detection accuracy:', ba.result().numpy())
print('COVID image detection recall:', recall_result)
print('COVID image detection precision:', precision_result)
print('COVID image detection F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 559 samples
Epoch 1/8
559/559 [==============================] - 5s 9ms/sample - loss: 42.0662 - accuracy: 0.4812
Epoch 2/8
559/559 [==============================] - 5s 9ms/sample - loss: 11.0056 - accuracy: 0.5206
Epoch 3/8
559/559 [==============================] - 5s 9ms/sample - loss: 6.8943 - accuracy: 0.5456
Epoch 4/8
559/559 [==============================] - 5s 9ms/sample - loss: 4.2934 - accuracy: 0.5707
Epoch 5/8
559/559 [==============================] - 5s 9ms/sample - loss: 2.3827 - accuracy: 0.5760
Epoch 6/8
559/559 [==============================] - 5s 9ms/sample - loss: 1.4371 - accuracy: 0.5581
Epoch 7/8
559/559 [==============================] - 5s 9ms/sample - loss: 0.8006 - accuracy: 0.6780
Epoch 8/8
559/559 [==============================] - 5s 9ms/sample - loss: 0.6454 - accuracy: 0.7120
COVID image detection accuracy: 0.7058824
COVID image detection recall: 0.45454547
COVID image detection precision: 0.7291667
COVID image detection F1 score: 0.560000018

In [14]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False

C:\Users\bruce\miniconda3\envs\capstone\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [15]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(500, activation='relu')
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    dense_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
model.fit(X_train, y_train, batch_size=32, epochs=8, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()
print('COVID image detection accuracy:', ba.result().numpy())
print('COVID image detection recall:', recall_result)
print('COVID image detection precision:', precision_result)
print('COVID image detection F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 559 samples
Epoch 1/8
559/559 [==============================] - 13s 24ms/sample - loss: 0.6677 - accuracy: 0.5921
Epoch 2/8
559/559 [==============================] - 12s 22ms/sample - loss: 0.5194 - accuracy: 0.7549
Epoch 3/8
559/559 [==============================] - 12s 21ms/sample - loss: 0.4746 - accuracy: 0.7925
Epoch 4/8
559/559 [==============================] - 12s 21ms/sample - loss: 0.4179 - accuracy: 0.8283
Epoch 5/8
559/559 [==============================] - 12s 21ms/sample - loss: 0.4094 - accuracy: 0.8301
Epoch 6/8
559/559 [==============================] - 12s 21ms/sample - loss: 0.3576 - accuracy: 0.8623
Epoch 7/8
559/559 [==============================] - 12s 21ms/sample - loss: 0.3251 - accuracy: 0.8712
Epoch 8/8
559/559 [==============================] - 12s 21ms/sample - loss: 0.3204 - accuracy: 0.8819
COVID image detection accuracy: 0.47058824
COVID image detection recall: 0.9480519
COVID image detection precision: 0.43452382
COVID image detection F1 sco

In [17]:
base_model = tf.keras.applications.ResNet50V2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False


In [18]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(500, activation='relu')
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    dense_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
model.fit(X_train, y_train, batch_size=32, epochs=8, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()
print('COVID image detection accuracy:', ba.result().numpy())
print('COVID image detection recall:', recall_result)
print('COVID image detection precision:', precision_result)
print('COVID image detection F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 559 samples
Epoch 1/8
559/559 [==============================] - 19s 34ms/sample - loss: 0.6717 - accuracy: 0.5635
Epoch 2/8
559/559 [==============================] - 17s 31ms/sample - loss: 0.5334 - accuracy: 0.7281
Epoch 3/8
559/559 [==============================] - 17s 31ms/sample - loss: 0.4463 - accuracy: 0.7979
Epoch 4/8
559/559 [==============================] - 17s 31ms/sample - loss: 0.4028 - accuracy: 0.8336
Epoch 5/8
559/559 [==============================] - 17s 31ms/sample - loss: 0.3628 - accuracy: 0.8676
Epoch 6/8
559/559 [==============================] - 17s 31ms/sample - loss: 0.3887 - accuracy: 0.8301
Epoch 7/8
559/559 [==============================] - 17s 31ms/sample - loss: 0.3105 - accuracy: 0.8962
Epoch 8/8
559/559 [==============================] - 18s 32ms/sample - loss: 0.2974 - accuracy: 0.9088
COVID image detection accuracy: 0.4117647
COVID image detection recall: 1.0
COVID image detection precision: 0.4117647
COVID image detection F1 score: 0.58

In [20]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

base_model = tf.keras.applications.VGG16(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(500, activation='relu')
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    dense_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train, batch_size=32, epochs=8, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()
print('COVID image detection accuracy:', ba.result().numpy())
print('COVID image detection recall:', recall_result)
print('COVID image detection precision:', precision_result)
print('COVID image detection F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 559 samples
Epoch 1/8
559/559 [==============================] - 17s 30ms/sample - loss: 1.7712 - accuracy: 0.5098
Epoch 2/8
559/559 [==============================] - 17s 30ms/sample - loss: 0.9032 - accuracy: 0.6583
Epoch 3/8
559/559 [==============================] - 16s 29ms/sample - loss: 0.6131 - accuracy: 0.7424
Epoch 4/8
559/559 [==============================] - 16s 29ms/sample - loss: 0.4871 - accuracy: 0.7925
Epoch 5/8
559/559 [==============================] - 16s 29ms/sample - loss: 0.3864 - accuracy: 0.8318
Epoch 6/8
559/559 [==============================] - 16s 29ms/sample - loss: 0.3264 - accuracy: 0.8587
Epoch 7/8
559/559 [==============================] - 17s 30ms/sample - loss: 0.2706 - accuracy: 0.8909
Epoch 8/8
559/559 [==============================] - 17s 30ms/sample - loss: 0.2277 - accuracy: 0.9088
COVID image detection accuracy: 0.7914438
COVID image detection recall: 0.77922076
COVID image detection precision: 0.73170733
COVID image detection F1 sco